In [7]:
import geopandas as gpd
#import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv("/home/mehdi/Documents/Transit/BusStop/20220111/stops.txt")

In [3]:
df

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,12,2201,MARKET + FAMILY DOLLAR,NaN,35.050703,-85.309532,NaN,NaN,NaN,NaN,NaN,2
1,17,2164,MARKET + 11TH,NaN,35.044250,-85.309435,NaN,NaN,NaN,NaN,NaN,2
2,18,3074,MARKET + 12TH,NaN,35.042347,-85.309070,NaN,NaN,NaN,NaN,NaN,2
3,19,2204,MARKET + COWART,NaN,35.040952,-85.308387,NaN,NaN,NaN,NaN,NaN,2
4,21,2217,MARKET + MAIN,NaN,35.036378,-85.307322,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1173,2339,1873,Harper & Bell-0,NaN,35.068415,-85.313000,NaN,NaN,NaN,NaN,NaN,2
1174,2340,1874,Ziegler & May-0,NaN,35.067780,-85.312020,NaN,NaN,NaN,NaN,NaN,2
1175,2341,2802,Frazier & Woodland,NaN,35.062620,-85.307630,NaN,NaN,NaN,NaN,NaN,2
1176,2342,2805,Cherokee & Spring,NaN,35.067020,-85.314890,NaN,NaN,NaN,NaN,NaN,2


In [4]:
LAT = df['stop_lat']
LON = df['stop_lon']

#Check the environment variable SUMO_HOME is set before running the script
#The <SUMO_HOME>/tools directory must be on the python load path
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")
# Connect to SUMO  
import traci
traci.start(["sumo", "-n", "Chattanooga_SUMO_Network.net.xml"])


#Convert geographic coordinates to position info (edge ID, lane position and lane index)
def get_edge(lon, lat):
    edgeID, lanePosition, laneIndex = traci.simulation.convertRoad(lon,lat, True)
    return edgeID, lanePosition, laneIndex

edgeinf =[]
for i in range(len(LAT)):
    edgeinf.append(get_edge(LON[i],LAT[i]))
# Write to     
df_bus = pd.DataFrame(edgeinf, columns=['edgeID', 'lanepos', 'laneind'])
df_bus.to_excel(r'busstops.xlsx', index = False)
result = pd.concat([df, df_bus], axis=1, join="inner")
result.to_excel('BusstopComplete.xlsx',index = False)

 Retrying in 1 seconds


# Create bus stop additional xml

In [17]:
# for testing
if os.path.exists('busStopsCARTA.add.xml'):
    os.remove("busStopsCARTA.add.xml")

In [18]:
#Read xlsx file from folder named "data"
data = pd.read_excel("BusstopComplete.xlsx")

#Create new columns 'startpos' and 'endpos' in data based on the "lanepos" in "busstops.xlsx"
data['startPos'] = round(data["lanepos"] - 5, 2)
data['endPos'] = round(data['startPos'] + 5, 2)
data['startPos'][data['startPos']<0] = 0
#Create busStopsCARTA.txt file to write in
f = open("busStopsCARTA.add.xml", "x")
#Write the fist line in the .txt file
f.write("<additional>\n")

# helper function to parse data to html
def parser(r):
    return '\t<busStop id="busStop_' + r['edgeID']+ "_"\
        + r['laneind'] + "_" + r['stop_id']\
        + '" lane="' + r['edgeID'] + '_'\
        + r['laneind'] + '" startPos="'\
        + r['startPos'] + '" endpos="'\
        + r['endPos'] + '" friendlyPos="1"/>\n'
#Write all the bus stop defination under the "<additional>" 
data["export"] = data.apply(lambda x: parser(x.astype(str)), axis=1)
for index, row in data.iterrows():
    f.write(row['export'])

#Write the last line in the .txt file
f.write("</additional>")
f.close()

<ipython-input-18-914fd54c72a2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['startPos'][data['startPos']<0] = 0
